In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px 
from plotly.offline import iplot

from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from imblearn.combine import SMOTETomek

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

import warnings


# set Default
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.2f}'.format
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = "warn"

# Used Color
used_color = ["#ADA2FF", "#C0DEFF", "#FCDDB0", "#FF9F9F", "#EDD2F3"]

In [1]:
def update_layout(
    title_font_size = 28,
    hover_font_size = 16,
    hover_bgcolor = '#111',
    showlegend = False
):
    fig.update_layout(
        showlegend = showlegend,
        title = {
            'font' : {
                'size' : title_font_size,
                'family' : '<b>poppins'
            }
        },
        
        hoverlabel = {
            'bgcolor' : hover_bgcolor,
            'font_size' : hover_font_size,
            'font_family' : 'poppins'
        }
    )

In [2]:
def count_viz_finc(
    data_frame,
    column_name,
    title = 'Chart Title',
    title_font_size = 30,
    x_label = 'X',
    y_label = 'Y',
    showlegend = False,
    hover_template = 'None'
):
    value_counts = df[colum_name].value_counts(normalize=1)*100

SyntaxError: expected ':' (4091340544.py, line 3)